# KLUE Relation Extraction - 어플리케이션 구조 설계

이 노트북은 **데이터 → 전처리 → 학습 → 평가 → 추론**까지의 전체 흐름과 모듈 구성을 설계합니다.
LLM 선택은 다음 단계에서 진행합니다.

## 1. 목표 및 범위
- KLUE RE 데이터셋 기반 관계 분류 태스크
- 고정 라벨(30개) 분류 문제
- 학습/평가/추론 파이프라인을 일관된 구조로 설계

## 2. 시스템 개요(High-Level)

```
Raw CSV
  │
  ├─ Data Loader
  │     └─ Schema Validation
  │
  ├─ Preprocess / Feature Builder
  │     └─ Entity Marker Insertion
  │
  ├─ Model
  │     └─ Encoder + Classification Head
  │
  ├─ Train / Eval
  │     └─ Metrics (Macro F1, Micro F1)
  │
  └─ Inference
        └─ Text + Entities → Relation Label
```

## 3. 모듈 구성

### 3.1 Data Layer
- 입력: `sentence`, `subject_entity`, `object_entity`, `label`, `source`
- 역할: 데이터 로드, 스키마 확인, train/valid 분리 확인

### 3.2 Preprocess Layer
- 엔티티 span 검증
- 입력 포맷 생성(예: `[SUBJ]...[/SUBJ]`, `[OBJ]...[/OBJ]`)
- 라벨 매핑(id ↔ label)

### 3.3 Model Layer
- 입력 텍스트 → 토크나이즈 → 인코더 → 분류 헤드
- 모델 구조는 LLM 선정 단계에서 확정

### 3.4 Train/Eval Layer
- Loss: Cross Entropy
- Metric: Macro F1 중심(클래스 불균형 고려)
- 체크포인트/로그 관리

### 3.5 Inference Layer
- 입력: 문장 + 엔티티 정보
- 동일 전처리 적용 후 라벨 예측

## 4. 데이터 흐름 설계

### 입력 → 전처리 → 학습
1) CSV 로드
2) 엔티티 파싱 및 span 검증
3) 입력 포맷 변환(마커 삽입)
4) 라벨 매핑
5) 학습/평가

### 추론
1) 사용자 입력 문장 + 엔티티 정보 수집
2) 동일 전처리 적용
3) 모델 예측 → 라벨 반환

## 5. 입출력 정의

### 학습 입력
- `input_text`: 엔티티 마킹된 문장
- `label_id`: 정수 라벨

### 추론 입력
- `sentence`
- `subject_entity` (word, start_idx, end_idx, type)
- `object_entity` (word, start_idx, end_idx, type)

### 추론 출력
- `pred_label`
- (optional) `confidence`

## 6. 전처리 규칙(요약)
- 엔티티 span은 문장과 정확히 일치해야 함
- 마커 삽입 시 뒤에서부터 처리하여 인덱스 꼬임 방지
- 라벨 매핑은 고정(0~29)

## 7. 로깅/재현성
- Seed 고정
- 실험 설정(config) 기록
- 학습 로그 저장 (metric, loss)

## 8. 리스크 및 대응
- **클래스 불균형**: macro F1 기준 평가, 추후 가중치/샘플링 도입
- **중복 샘플 존재**: 필요 시 중복 제거/하향 가중치
- **긴 문장**: 토큰 길이 제한 고려(Truncation 전략)

## 9. 다음 단계
- LLM 후보 비교 및 선정
- 모델 세부 구조 확정
- 베이스라인 학습 및 평가

In [ ]:
# 파이프라인 설정 예시
config = {
    "seed": 42,
    "max_length": 256,
    "batch_size": 16,
    "lr": 2e-5,
    "epochs": 3,
    "metric": "macro_f1",
}
print(config)